train dataset

In [4]:
import os
import cv2
import xml.etree.ElementTree as ET

# Paths
image_dir = '/home/saku/2010776109_RUSHO/RSOD_YOLO-20250708T170752Z-1-001/RSOD_YOLO/val/images'
label_dir = '/home/saku/2010776109_RUSHO/RSOD_YOLO-20250708T170752Z-1-001/RSOD_YOLO/val/labels'
output_dir = '/home/saku/2010776109_RUSHO/RSOD_YOLO-20250708T170752Z-1-001/RSOD_YOLO/val/Annotations'

os.makedirs(output_dir, exist_ok=True)

# Class names — must match your dataset
classes = ['aircraft', 'oiltank', 'overpass', 'playground']

for img_file in sorted(os.listdir(image_dir)):
    if not img_file.endswith(('.jpg', '.png', '.jpeg')):
        continue

    img_path = os.path.join(image_dir, img_file)
    label_path = os.path.join(label_dir, os.path.splitext(img_file)[0] + '.txt')
    img = cv2.imread(img_path)
    h, w, c = img.shape

    # Make root
    annotation = ET.Element('annotation')
    ET.SubElement(annotation, 'folder').text = os.path.basename(image_dir)
    ET.SubElement(annotation, 'filename').text = img_file
    ET.SubElement(annotation, 'path').text = img_path

    size = ET.SubElement(annotation, 'size')
    ET.SubElement(size, 'width').text = str(w)
    ET.SubElement(size, 'height').text = str(h)
    ET.SubElement(size, 'depth').text = str(c)

    ET.SubElement(annotation, 'segmented').text = '0'

    if os.path.exists(label_path):
        with open(label_path) as f:
            for line in f:
                cls_id, xc, yc, bw, bh = map(float, line.strip().split())
                cls_name = classes[int(cls_id)]
                x1 = int((xc - bw/2) * w)
                y1 = int((yc - bh/2) * h)
                x2 = int((xc + bw/2) * w)
                y2 = int((yc + bh/2) * h)

                obj = ET.SubElement(annotation, 'object')
                ET.SubElement(obj, 'name').text = cls_name
                ET.SubElement(obj, 'pose').text = 'Unspecified'
                ET.SubElement(obj, 'truncated').text = '0'
                ET.SubElement(obj, 'difficult').text = '0'

                bndbox = ET.SubElement(obj, 'bndbox')
                ET.SubElement(bndbox, 'xmin').text = str(max(0, x1))
                ET.SubElement(bndbox, 'ymin').text = str(max(0, y1))
                ET.SubElement(bndbox, 'xmax').text = str(min(w, x2))
                ET.SubElement(bndbox, 'ymax').text = str(min(h, y2))

    # Write XML
    tree = ET.ElementTree(annotation)
    xml_file = os.path.join(output_dir, os.path.splitext(img_file)[0] + '.xml')
    tree.write(xml_file)

print(f"✅ Converted {len(os.listdir(output_dir))} images to Pascal VOC XMLs!")


✅ Converted 187 images to Pascal VOC XMLs!


cell 1: imports